In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import pulp
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, LpMinimize

In [2]:
# Assuming FoodDatabase.csv is in the same directory as your Jupyter Notebook
file_path = 'FoodDatabase.csv'
#All numbers (macros and micros) in the table are a serving of 100g

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Filter rows with nonzero 'Price / kg'
df_filtered = df[df['Price / kg'] != 0]

# Add 'Cost ($)' column with initial value of zero at the start
df_filtered.insert(0, 'Cost ($)', 0)

# Add Quantities' (g)' column with initial value of zero as the second column
df_filtered.insert(1, 'Quantities (g)', 0)

# Display the first few rows of the DataFrame to verify it was read correctly
df_filtered.info()
df_filtered.head()


#We want to minimize c_1 * x_1 + c_2 * x_2 ... c_n * x_n , where c is the cost per unit mass of a given food, and x is the mass of the given food (aka the objective function)

# The constraints are the macros and micros of a diet, and other conditions we set and must be met. 

#Excel was using Simplex LP method


<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 187 to 14168
Columns: 106 entries, Cost ($) to Comments
dtypes: float64(85), int64(15), object(6)
memory usage: 21.7+ KB


,Cost ($),Quantities (g),ID,name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),...,Caffeine (mg),Theobromine (mg),200 Calorie Weight (g),Sugar (g),Price / kg,Protein Quality,Product Name,Last Updated,Source,Comments
187,0,0,168263,Pork Fresh Loin Center Loin (Chops) Boneless S...,Meats,122.81,3.09,23.75,0.00,0.00,...,0,0,162.602,0.000,5.158811,1.0,Smithfield Fresh Pork Center Cut Loin Boneless...,01-Feb-24,https://www.walmart.com/ip/Smithfield-Fresh-Po...,0
482,0,0,168877,Rice White Long-Grain Regular Raw Enriched,Grains and Pasta,349.06,0.66,7.13,79.95,0.12,...,0,0,54.795,0.060,1.228000,0.0,"Great Value Long Grain Enriched Rice, 20 lb USA",01-Feb-24,https://www.walmart.com/ip/Great-Value-Lentils...,0
505,0,0,168894,Wheat Flour White All-Purpose Enriched Bleached,Grains and Pasta,344.58,0.98,10.33,76.31,0.27,...,0,0,54.945,0.000,1.080264,0.0,"Great Value All-Purpose Flour, 5LB Bag USA",24-Feb-24,https://www.walmart.com/ip/Great-Value-All-Pur...,0
516,0,0,169736,Pasta Dry Enriched,Grains and Pasta,351.63,1.51,13.04,74.67,2.67,...,0,0,53.908,0.350,1.999590,0.0,"Great Value Elbows, 48 oz PASTA USA",01-Feb-24,https://www.walmart.com/ip/Great-Value-Elbows-...,"not worth it in the US, dairy better, pricey"
525,0,0,170379,Broccoli,Vegetables,30.77,0.37,2.82,6.64,1.70,...,0,0,588.235,0.835,2.513267,1.0,"Great Value Frozen Broccoli Cuts, 32 oz Steama...",01-Feb-24,https://www.walmart.com/ip/Great-Value-Frozen-...,0


In [31]:
# Define the model
maintenance_model = LpProblem(name="Maintenance_Diet_Minimum_Cost", sense=LpMinimize)

# Define the decision variables (what we are solving for; quantities of each food)
x = {i: LpVariable(name=f"x_{i}", lowBound=0) for i in range(1, len(df_filtered)+1)}

# Arrays per column extracted from table
dollars_per_kg = df_filtered.iloc[:, 100].values # get array of dollars per kg per food
name_per_food = df_filtered.iloc[:, 3].values # get array of calories values per food
calories_per_100g = df_filtered.iloc[:, 5].values # get array of calories values per food


maintenance_model += lpSum(x[i] * (dollars_per_kg[i-1] / 1000) for i in range(1, len(df_filtered)+1))

#Defining Constraints

#Calorie Constraint

maintenance_calories = 2936 # set maintenance calories
maintenance_model += lpSum(x[i] * (calories_per_100g[i-1] / 100) for i in range(1, len(df_filtered)+1)) == maintenance_calories


print(maintenance_model)


#array of name per food



# Solve the optimization problem
status = maintenance_model.solve()

# Get the results and format them
print(f"status: {maintenance_model.status}, {LpStatus[maintenance_model.status]}")
monthly_cost = round(maintenance_model.objective.value() * 30, 2)
objective_value = round(maintenance_model.objective.value(), 2)
print(f"TOTAL COST OF DAILY DIET CALCULATED: ${objective_value}")

# Calculate and print the monthly diet cost

print(f"TOTAL COST OF MONTHLY DIET CALCULATED: ${monthly_cost}")

print("CONSISTING OF THE FOLLOWING FOODS:")
for i, var in x.items():
    var_value = int(round(var.value(), 0))  # Convert to int to remove decimal point
    if var_value > 0:
        print(f"{name_per_food[i-1]}: {var_value} g")  # Print food name and quantity in grams

print("\nError per constraint equation:")
for name, constraint in maintenance_model.constraints.items():
    print(f"{name}: {constraint.value()}")

Maintenance_Diet_Minimum_Cost:
MINIMIZE
0.0051588108*x_1 + 0.0034612533999999998*x_10 + 0.001080464203*x_11 + 0.00254*x_12 + 0.002314851*x_13 + 0.0043871938000000004*x_14 + 0.0010802638*x_15 + 0.0029541908*x_16 + 0.0065256752*x_17 + 0.0028219136*x_18 + 0.0029982832*x_19 + 0.001228*x_2 + 0.0020683344*x_20 + 0.0025242898999999997*x_21 + 0.0025132668*x_22 + 0.0033510224*x_23 + 0.0016424419*x_24 + 0.00170196664*x_25 + 0.0041667318*x_26 + 0.0010802638*x_3 + 0.00199959034*x_4 + 0.0025132668*x_5 + 0.0021825738*x_6 + 0.0018245131029999999*x_7 + 0.0058201968*x_8 + 0.005253863135*x_9 + 0.0
SUBJECT TO
_C1: 1.2281 x_1 + 1.1407 x_10 + 0.3404 x_11 + 1.3871 x_12 + 0.8918 x_13
 + 1.0828 x_14 + 0.4821 x_15 + 3.1866 x_16 + 1.0965 x_17 + 3.3885 x_18
 + 2.8462 x_19 + 3.4906 x_2 + 2.5225 x_20 + 4.5296 x_21 + 0.80726 x_22
 + 0.786126 x_23 + 0.690838 x_24 + 0.726571 x_25 + 0.768788 x_26 + 3.4458 x_3
 + 3.5163 x_4 + 0.3077 x_5 + 0.33 x_6 + 0.3681 x_7 + 1.2632 x_8 + 2.2512 x_9
 = 2936

VARIABLES
x_1 Continuous

In [26]:
name_per_food = df_filtered.iloc[:, 3].values # get array of calories values per food

print (name_per_food)

['Pork Fresh Loin Center Loin (Chops) Boneless Separable Lean Only Raw'
 'Rice White Long-Grain Regular Raw Enriched'
 'Wheat Flour White All-Purpose Enriched Bleached' 'Pasta Dry Enriched'
 'Broccoli' 'Carrots' 'Canned Tomato Puree' 'Nonfat American Cheese'
 'Cheese Spread American Or Cheddar Cheese Base Reduced Fat'
 'Chicken Liver All Classes Raw' 'Skim Milk' 'Eggs (Raw)'
 'Chicken Gizzard All Classes Raw'
 'Chicken Broilers Or Fryers Breast Skinless Boneless Meat Only With Added Solution Raw'
 'Apples' 'Lentils Raw'
 'Chicken Dark Meat Thigh Meat Only With Added Solution Raw'
 'Quaker Quick Oats Dry' 'Beans Navy Mature Seeds Raw' 'White Bread'
 'Popcorn Ready-To-Eat Packaged Unsalted'
 'Whole Chicken Raw (51.52% of Chicken Stewing Meat Only Raw), 2.16% of liver,heart,gizzard each'
 'Chicken Thigh (accounting for debone and deskin) , 66% of Chicken Dark Meat Thigh Meat Only With Added Solution Raw'
 'Chicken Drumstick (accounting for debone and deskin) , 58% of Chicken Dark Meat Thi